In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.DataFrame({"A": [1,2,3], "B":["ab","cd","dffds"]})

In [ ]:
penguins_df = pd.read_csv('/kaggle/input/palmer-archipelago-antarctica-penguin-data/penguins_size.csv')

In [ ]:
penguins_df.shape

In [ ]:
# pd.Series

In [ ]:
penguins_df['body_mass_g'].mean()

In [ ]:
penguins_df.loc[221]

In [ ]:
species_mapping = {sp:i for i, sp in enumerate(penguins_df['species'].unique())}

In [ ]:
species_mapping

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
penguins_train, penguins_test = train_test_split(penguins_df, test_size=0.2)

In [ ]:
penguins_train

In [ ]:
sns.scatterplot(data=penguins_train, x='culmen_length_mm', y='culmen_depth_mm', hue='species')

In [ ]:
sns.scatterplot(data=penguins_train, x='culmen_length_mm', y='culmen_depth_mm', hue='sex')

In [ ]:
sns.distplot(a=penguins_df['culmen_length_mm'])

In [ ]:
sns.pairplot(data=penguins_train, hue='sex')

In [ ]:
from collections import Counter

In [ ]:
for sp in penguins_train['species'].unique():
    print(sp)
    subdf = penguins_train[penguins_train.species==sp]
    print(Counter(subdf.sex,))
#     print(np.unique(subdf.sex, return_counts=True))

In [ ]:
penguins_train.sample(6)

In [ ]:
penguins_train.sex.unique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [ ]:
penguins_train.columns

In [ ]:
def prepare_data(penguins, pipeline_stages=None, use_dummy=False):
    penguins = penguins.copy()
    penguins = penguins[penguins.sex != '.']
    penguins.dropna(axis=0, inplace=True)
    
    categories_for_one_hot = ['species', 'island']
    if not pipeline_stages:
        if use_dummy:
            one_hot = OneHotEncoder(sparse=False,drop='first')
        else:
            one_hot = OneHotEncoder(sparse=False)
        one_hot.fit(penguins.loc[:,categories_for_one_hot])
    else:
        one_hot = pipeline_stages['one_hot']
        
    one_hot_result = one_hot.transform(penguins.loc[:,categories_for_one_hot])
    
    offset = 0
    print(one_hot.categories_)
    for category_values, category_name in zip(one_hot.categories_, categories_for_one_hot):
        if use_dummy:
            category_values = category_values[1:]
        for  value in category_values:
            column_name = f'{category_name}={value}'
            penguins[column_name] = one_hot_result[:,offset]
            offset += 1
            
    
    
    penguins.drop(['species', 'island'], axis=1, inplace=True)
    
    if not pipeline_stages:
        sex_encoder = LabelEncoder()
        sex_encoder.fit(penguins['sex'])
    else:
        sex_encoder = pipeline_stages['sex_encoder']
    
    sex_column = sex_encoder.transform(penguins['sex'])
    
    size_features = ['culmen_length_mm', 'culmen_depth_mm','flipper_length_mm', 'body_mass_g']
    
    if not pipeline_stages:
        scaler = StandardScaler().fit(penguins.loc[:,size_features].values)
    else:
        scaler = pipeline_stages['scaler']
    
    scaled_features = scaler.transform(penguins.loc[:,size_features].values)
    print(scaled_features)
    penguins.loc[:,size_features] = scaled_features
    
    penguins.drop(['sex'], axis=1, inplace=True)
    
    
    
    
    print(penguins.columns)
    
    return penguins.values, sex_column, {"one_hot": one_hot, "sex_encoder": sex_encoder, "feature_names": penguins.columns, "scaler": scaler}
    

In [ ]:
X_train, y_train, pipeline = prepare_data(penguins_train, use_dummy=True)

In [ ]:
X_train.shape

In [ ]:
X_train[:4]

In [ ]:
pipeline

In [ ]:
pipeline['scaler'].mean_

In [ ]:
X_train[:4]

In [ ]:
X_train.std(axis=0)

In [ ]:
y_train

In [ ]:
X_test, y_test, _ = prepare_data(penguins_test, pipeline, use_dummy=True)

In [ ]:
y_test

In [ ]:
X_test.shape

In [ ]:
X_test

In [ ]:
X_test.mean(axis=0)

In [ ]:
X_test.std(axis=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(max_depth=3)

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
y_test

In [ ]:
y_pred = dt.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
pd.DataFrame(metrics.confusion_matrix(y_test, y_pred), index=pipeline['sex_encoder'].classes_, columns=pipeline['sex_encoder'].classes_)

In [ ]:
metrics.accuracy_score(y_test,y_pred)

In [ ]:
print(metrics.classification_report(y_test, y_pred, target_names=pipeline['sex_encoder'].classes_))

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
graph = export_graphviz(dt,feature_names=pipeline['feature_names'], class_names=pipeline['sex_encoder'].classes_)

In [ ]:
!pwd

In [ ]:
with open('graph.dot','w') as f:
    f.write(graph)

In [ ]:
!dot -Tpng graph.dot -o tree.png 

In [ ]:
from IPython.display import Image 


In [ ]:
Image("./tree.png")

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression().fit(X_train, y_train)

In [ ]:
lpred = clf.predict(X_test)

In [ ]:
pd.DataFrame(metrics.confusion_matrix(y_test, lpred), index=pipeline['sex_encoder'].classes_, columns=pipeline['sex_encoder'].classes_)

In [ ]:
clf.coef_

In [ ]:
pd.DataFrame({"weight": clf.coef_[0]}, index=pipeline["feature_names"])

In [ ]:
print(clf.intercept_)

In [ ]:
graph